# Google Scrape Counter

##### '''
Authors: BarendL + BruceB

Version: 1.1.6 (6 Dec 2017) (Set up for Djibouti pull)

Description:  Pulls all URLS from a specified number of pages from a specific google search. 
For best results use a google search on a specific site (like site:www.news24.com) 
you can also specify the dates, etc. The program then pulls the titles, dates and 
the bodies from each of the url’s. Finally it does a keyword search on each item
identifying how many times specified keywords were mentioned. 

Future Expansion:
Will weight keywords giving article score

'''

In [1]:
from bs4 import BeautifulSoup as bs
import urllib.parse
import urllib.request
import re
import pandas as pd
import requests
import random
import csv
from time import sleep
#import tldextract
import glob
import pylab
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import datetime
from random import randint
import re
from tkinter import *
import tkinter.messagebox

# Functions

In [2]:
def GoogleAddress (website, key_word, begin_date = '01/01/2016', end_date = '01/01/2017'):
    '''
    Returns the base Google url that can be used for scraping.
    
    website -> web address in form www.?.suffix or ?.suffix (string)
    key_word -> the keyword you want to search for, can use logcal operators for multiple keywords (string)
    begin_date -> start of your search datarange (MM/DD/YYY)(string)
    end_date -> end of your search datarange (MM/DD/YYY) (string)
    
    >>>  GoogleAddress ('ann7.com', 'bitcoin','8/1/2017','9/30/2017')+str(a)+end_url
    'https://www.google.co.za/search?q=bitcoin+site:ann7.com&num=100&safe=off&hl=en&dcr=0&tbs=cdr:1,cd_min:8/1/2017,cd_max:9/30/2017&ei=LmbkWaqbA8b9mAHy76_wAg&start=0&sa=N&biw=1706&bih=1006'
    
    '''
    base_url = 'https://www.google.co.za/search?q='+(key_word)+'+site:'+(website)+'&num=100&safe=off&hl=en&dcr=0&tbs=cdr:1,cd_min:'+begin_date+',cd_max:'+end_date+'&ei=LmbkWaqbA8b9mAHy76_wAg&start='
    return base_url

In [3]:
GoogleAddress?

In [3]:
GoogleAddress('www.news24.com','rabbit','01/05/2016')

'https://www.google.co.za/search?q=rabbit+site:www.news24.com&num=100&safe=off&hl=en&dcr=0&tbs=cdr:1,cd_min:01/05/2016,cd_max:01/01/2017&ei=LmbkWaqbA8b9mAHy76_wAg&start='

In [4]:
def CSV_Pull_NoCopy (csv_name, copy_column_name):
    '''
    Get CSV Data rather than Google URLs. This will remove copies in a specific column. In gdelt use SOURCEURL
        
    csv_name -> The csv you want to pull - 'name.csv'
    copy_column_name -> The name of the column that you want to check for duplicates - 'column'
    
    >>>  df = CSV_Pull_NoCopy('name.csv','column')
    Creates dataframe from name csv and removes all duplicates from the column called column 
    '''

    #df = pd.read_csv(csv_name,low_memory=False, encoding='ISO-8859-1')
    df = pd.read_csv(csv_name,low_memory=False, encoding='UTF-8')
    df_urls = df.drop_duplicates(subset=[copy_column_name])
    return df_urls

In [5]:
def CSV_Pull_WithCopy (csv_name):
    '''
    See CSV_Pull_NoCopy - This does the same just without removing copies of URL's
    '''

    df_urls = pd.read_csv(csv_name,low_memory=False, encoding='UTF-8')
    return df_urls

In [6]:
def get_keywords(keywords_file):
    with open(keywords_file, 'r') as f:
        reader = csv.reader(f)
        keywords = list(reader)[0]
        return keywords

In [7]:
#Test Functions for GUI
def doNothing():
    print("Nothing done")
    
def mstore(text):
    testlist = text
    print (testlist)
    return testlist

In [1]:
print?

# Collecting User Input

In [9]:
#Getting the input from the USER

#Will use the name to create ID for CSV name
name = input ("What is your name: ")
print ("Hello "+ name+" \nWelcome to the Awesome Google scraper,\n please fill in the following questions:")

#This is the info you need to create a google search
key = input ("What keyword/s are you looking for (Seperate with OR if multiple): ")
www = input ("What website do you want to search in (Seperate with OR site: if multiple)(Note - Works better one at a time): ")
beg = input ("What is the start of your search datarange (MM/DD/YYY): ")
end = input ("What is the end of your search datarange (MM/DD/YYY): ")
end_url = '&sa=N&biw=1706&bih=1006'
e_url = input ("Use default end URL: '"+end_url+"' (y/n): ")
if e_url == 'y':
    print ('Using default end URL')
elif e_url == 'n':
    end_url = input ("What end URL do you want to use: ")
else:
    print ("Do not be silly, enter y/n; using default end url for now")
a = input ("What page do you want to start seaching on (0 = P1, 10 = P2, 20 = P3, etc): ")
test_url = GoogleAddress (www, key,beg,end)+str(a)+end_url
print ("Please test this URL in your browser (If you used OR, make sure you select complete URL):\n"+test_url)

base_url = GoogleAddress (www, key,beg,end)

What is your name: df
Hello df 
Welcome to the Awesome Google scraper,
 please fill in the following questions:
What keyword/s are you looking for (Seperate with OR if multiple): ramaphosa
What website do you want to search in (Seperate with OR site: if multiple)(Note - Works better one at a time): www.news24.com
What is the start of your search datarange (MM/DD/YYY): 10/20/2017
What is the end of your search datarange (MM/DD/YYY): 20/20/2017
Use default end URL: '&sa=N&biw=1706&bih=1006' (y/n): y
Using default end URL
What page do you want to start seaching on (0 = P1, 10 = P2, 20 = P3, etc): 0
Please test this URL in your browser (If you used OR, make sure you select complete URL):
https://www.google.co.za/search?q=ramaphosa+site:www.news24.com&num=100&safe=off&hl=en&dcr=0&tbs=cdr:1,cd_min:10/20/2017,cd_max:20/20/2017&ei=LmbkWaqbA8b9mAHy76_wAg&start=0&sa=N&biw=1706&bih=1006


In [9]:
end_url = "&sa=N&biw=1711&bih=1006"
base_url = "https://www.google.co.za/search?q=Chine+OR+Chinois+OR+Chinoise+site:www.rtd.dj+OR+site:http://www.presidence.dj+OR+site:http://www.lanationdj.com/&num=100&safe=off&hl=en&dcr=0&tbs=cdr:1,cd_min:1/1/2016,cd_max:11/6/2017&ei=tTwAWuW5BcbMjwPAyqZI&start="

In [11]:
url =base_url+'0'+end_url

In [12]:
url

'https://www.google.co.za/search?q=Chine+OR+Chinois+OR+Chinoise+site:www.rtd.dj+OR+site:http://www.presidence.dj+OR+site:http://www.lanationdj.com/&num=100&safe=off&hl=en&dcr=0&tbs=cdr:1,cd_min:1/1/2016,cd_max:11/6/2017&ei=tTwAWuW5BcbMjwPAyqZI&start=0&sa=N&biw=1711&bih=1006'

# GUI Test - In progess

In [119]:
# Building a GUI Test

root=Tk()
root.geometry("400x200")
root.title('Form Test')

# ********* The Main Menu ********

menu = Menu(root)
root.config(menu=menu)

subMenu = Menu(menu)
menu.add_cascade(label="File", menu=subMenu)
subMenu.add_command(label="New Project...", command=doNothing)
subMenu.add_command(label="New...", command=doNothing)
subMenu.add_separator()
subMenu.add_command(label="Exit", command = menu.quit)

editMenu = Menu(menu)
menu.add_cascade(label="Edit", menu=editMenu)
editMenu.add_command(label="Redo", command=doNothing)
#********************

#tkinter.messagebox.showinfo("Note","Please fill out all fields")

#base_url = GoogleAddress (www, key,beg,end)+str(a)+end_url
#print ("Please test this URL in your browser and make sure it is pulling the info you are looking for: "+base_url)
    
    
####Labels####    
label1 = Label(root, text = "Keyword: ")
label2 = Label(root, text = "Website: ")
label3 = Label(root, text = "Start Date (MM/DD/YYY): ")
label4 = Label(root, text = "End Date (MM/DD/YYY): ")


entry1 = Entry(root)
entry2 = Entry(root)
entry3 = Entry(root)
entry4 = Entry(root)

label1.grid(row=0, sticky=E)
label2.grid(row=1, sticky=E)
label3.grid(row=2, sticky=E)
label4.grid(row=3, sticky=E)

entry1.grid(row=0, column=1)
entry2.grid(row=1, column=1)
entry3.grid(row=2, column=1)
entry4.grid(row=3, column=1)

checkbox = Checkbutton(root, text = "Keep me logged in")
checkbox.grid(row=4,columnspan=2)

####Buttons####
savebutton=Button(root,text="Save",command = lambda: mstore(mEntryname.get()))
savebutton.grid(row=5, column=0)

exitbutton=Button(root,text="Exit",command=root.destroy)
exitbutton.grid(row=5, column=1)

###############

##########Message Boxes#####
answer = tkinter.messagebox.askquestion("Question 1", "Do you want to do nothing")

if answer == 'yes':
    print("Doing nothing...")
    #print("Congrats! "+ doNothing())
else:
    print("Doing something...")
############
    
root.mainloop()

Doing something...
Nothing done
Nothing done
Nothing done


# Google URL Pull

In [ ]:
# Send to Bruce (DO NOT USE)

"""
#The number below reprents how many pages it will page through *10 - so 200 will do 20 google pages. 
#Whatch out this will get your IP banned, google only alows something like 40 calls per hour
flag = 0
testend = '<li>Make sure that all words are spelled correctly.</li><li>Try different keywords.</li>'
page = requests.get(base_url+str(0)+end_url).content

urls = [];
one_url = ("")
n = 0
print(n)

while flag == 0:
    #page = requests.get(base_url+str(a)+end_url,proxies = Proxies).content
    
    soup = bs(page, "html.parser")
    
    links = soup.findAll(["cite"])
    for link in links:
        #print(link.text)
        one_url+=link.text+','
    
    urls.append(one_url)
    n+=10
    newpage = base_url+str(n)+end_url
    print(newpage)
    page = requests.get(newpage).content
#    print(base_url+str(a)+end_url)
    #Trying to confuse google with random numbers, this does not confuse them...
    sleep(2+random.random()*9)
    print(flag, n, str(n))
    
    if testend in str(page):

        flag = 1
"""

In [13]:
url = "https://www.google.co.za/search?q=Chine+OR+Chinois+OR+Chinoise+site%3Awww.rtd.dj+OR+site%3Ahttp%3A%2F%2Fwww.presidence.dj+OR+site%3Ahttp%3A%2F%2Fwww.lanationdj.com%2F&num=100&safe=off&hl=en&dcr=0&biw=1711&bih=1006&source=lnt&tbs=cdr%3A1%2Ccd_min%3A11%2F6%2F2017%2Ccd_max%3A12%2F6%2F2017&tbm="

In [14]:
counter = 0
myhappylist = []
page = requests.get(url).content
soup = bs(page, "html.parser")
# New Version  - Rather Search for all R's in h3 class. This is messy so you need to clean it up a bit with while loop
links = soup.findAll("h3", class_="r")
# while counter <= len(links):
#There are normally 100 urls on each google search page that is why 100 below
while counter <= 100:
    start = str(links[counter]).find('url?q=')
    end = str(links[counter]).find('&amp')
    test5 = str(links[counter])
    if start != -1:
        #print(start,end)
        test6 = test5[start+6:end]
        myhappylist.append(test6)
        print (test6)
    counter += 1

SSLError: HTTPSConnectionPool(host='www.google.co.za', port=443): Max retries exceeded with url: /search?q=Chine+OR+Chinois+OR+Chinoise+site%3Awww.rtd.dj+OR+site%3Ahttp%3A%2F%2Fwww.presidence.dj+OR+site%3Ahttp%3A%2F%2Fwww.lanationdj.com%2F&num=100&safe=off&hl=en&dcr=0&biw=1711&bih=1006&source=lnt&tbs=cdr%3A1%2Ccd_min%3A11%2F6%2F2017%2Ccd_max%3A12%2F6%2F2017&tbm= (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",),))

In [15]:
 #Whatch out this will get your IP banned, google only alows something like 40 calls per hour

#Emty variables for loops below
urls = [];
test = ("")

a = 0
counter = 0
start = ''
end = ''
test5 = ''
test6 = ''

myhappylist = []

# This is number of pages you want to search. The number (a) below reprents how many pages it will page through *10 - so 200 will do 20 google pages.
while a<40:
    #With Proxy - Safe
    #page = requests.get(base_url+str(a)+end_url,proxies = Proxies).content
    #Without Proxy - Fast
    page = requests.get(base_url+str(a)+end_url).content

    soup = bs(page, "html.parser")
    
#Old version: This Section will work on certain google searches but not all. Sometimes urls will come out with ... in the middle. Rather use the next version. 
    '''
    links = soup.findAll(["cite"])
    for link in links:
        print(link.text)
        test+=link.text+','
    '''
# New Version  - Rather Search for all R's in h3 class. This is messy so you need to clean it up a bit with while loop
    links = soup.findAll("h3", class_="r")
   # while counter <= len(links):
#There are normally 100 urls on each google search page that is why 100 below
    while counter <= 100:
        start = str(links[counter]).find('url?q=')
        end = str(links[counter]).find('&amp')
        test5 = str(links[counter])
        if start != -1:
            #print(start,end)
            test6 = test5[start+6:end]
            myhappylist.append(test6)
            print (test6)
        
        counter += 1
        #test6 = ''
     
    print ("page"+str(a)+" done")
    a=a+10
        #This part goes with the old cite version
    #urls.append(test)
    
    #Trying to confuse google with random numbers, this does not confuse them...
    sleep(2+random.random()*9)

SSLError: HTTPSConnectionPool(host='www.google.co.za', port=443): Max retries exceeded with url: /search?q=Chine+OR+Chinois+OR+Chinoise+site%3Awww.rtd.dj+OR+site%3Ahttp%3A%2F%2Fwww.presidence.dj+OR+site%3Ahttp%3A%2F%2Fwww.lanationdj.com%2F&num=100&safe=off&hl=en&dcr=0&biw=1711&bih=1006&source=lnt&tbs=cdr%3A1%2Ccd_min%3A11%2F6%2F2017%2Ccd_max%3A12%2F6%2F2017&tbm= (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",),))

In [25]:
len(links)

100

In [11]:
len(myhappylist)

65

In [12]:
urlist = []
for k in range(len(myhappylist)):
    urlist.append((myhappylist[k].split(',')))

In [13]:
flattened_urllist = [y for x in urlist for y in x]

In [14]:
df_urls = pd.DataFrame(flattened_urllist, columns = ['URL'])

In [17]:
df_urls.to_csv("RepublikeinTest.csv")

# Get URLs via CSV

In [20]:
#Run this if you want to use GDELT or other CSV with URLs rather than Google method above
#df_urls = GDELT_Pull_NoCopy('ZAPROTESTSEP17.csv', 'SOURCEURL')
df_urls = CSV_Pull_NoCopy(r'H:\Work\Projects\NK in Africa\Djib URLS\6novto6dec1.csv','URL')

In [21]:
df_urls

,URL
0,http://www.lanationdj.com/mega-port-de-classe-...
1,http://www.lanationdj.com/visite-president-ism...
2,http://www.lanationdj.com/president-guelleh-pr...
3,http://www.lanationdj.com/president-guelleh-de...
4,http://www.lanationdj.com/enjeux-vent-dest-ven...
5,http://www.lanationdj.com/aboubaker-omar-hadi-...
6,http://www.lanationdj.com/mahmoud-ali-youssouf...
7,http://www.lanationdj.com/djibouticccl-lor-bla...
8,http://www.lanationdj.com/president-de-republi...
9,http://www.presidence.dj/article.php?ID=479


In [22]:
len(df_urls)

41

In [22]:
len(df_key)

376

In [48]:
len(df_remove)

396

In [31]:
keywords = '|'.join(pullkeywords)

In [32]:
keywords

'ï»¿1049.fm;2oceansviberadio.com;3d-car-shows.com;4gafrica.co.za;5stardurban.co.za;6000.co.za;702.co.za;afkinsider.com;afribiz.info;africabusiness.com;africajournalismtheworld.com;africanbusinessreview.co.za;africanmining.com;africanreview.com;africansafarisguide.com;africaoutlookmag.com;afronline.org;alumnicareers.co.za;amabhungane.co.za;anc.org.za;ann7.com;appleza.co.za;artlink.co.za;artsmart.co.za;asherworldturns.com;bbqonline.co.za;bdlive.co.za;bizcommunity.com;biznews.com;biznisafrica.co.za;bookslive.co.za;businesstech.co.za;cansa.org.za;capetalk.co.za;carletonvilleherald.com;cbn.co.za;channel24.co.za;china-africa-reporting.co.za;citizen.co.za;citypress.co.za;cnbcafrica.com;constructionreviewonline.com;creamermedia.co.za;cxpress.co.za;da.org.za;dailymaverick.co.za;dailysun.co.za;deentv.co.za;defenceweb.co.za;destinyconnect.com;destinyman.com;disrupt-africa.com;dod.mil.za;drum.co.za;dstv.com;eatout.co.za;ecr.co.za;enca.com;engineeringnews.co.za;eprop.co.za;esi-africa.com;ewn.co.za;

In [23]:
#Make sure this gives you a URL, if it does not you probably need to select the correct column for URLs instead of 3
url = (df_urls.iloc[0][0])
print (url)

http://www.lanationdj.com/mega-port-de-classe-mondiale-cofinance-chine-a-ete-officiellement-inaugure-president-de-republique-ismail-omar-guelleh-mercredi-24-mai-dernier/


# Removing unwanted URLs with Keyword Search

In [19]:
testdf = df_urls

In [82]:
len(df_urls)

190

In [18]:
removewords = "cats"
#removewords ="doom pastor|abortion|accident|airways|andreas_spath|art|assisted suicide|athletics|beeld|blaze|box office|boxing|capsize|cricket|cycling|dante terrell smith|drown|equestrian|film festival|football|formula one|golf|gymnastics|heist|hockey|horn|horse racing|jayde panayiotou|judo|lethebo rabalago|lover|motorcycling|motorsports|olympics|oscar pistorius|pageant|rhino|rowing|rugby|rugbyu|sailing|senzo meyiwa|shack|shrien dewani|soccer|späth|sports|squash|stab|stransham-ford|surfing|swimming|tennis|triathlon|"
#removewords = "engineeringnews|entertainment|soccer|rugby|golf|abortion|athletics|boxing|capsize|cricket|cycling|equestrian|film festival|football|gymnastics|hockey|motorsports|olympics|sports|surfing|swimming|tennis|triathlon"

In [22]:
#Trusted Sources List 1, still need to split into 3 groups:
keywords = "www.iol.co.za|ewn.co.za|www.timeslive.co.za|www.dailymaverick.co.za|www.bbc.co.uk|citizen.co.za|www.aljazeera.com|allafrica.com|www.enca.com|www.news24.com|www.politicsweb.co.za|www.polity.org.za"
#ZA Sources:
#keywords = "https://businesstech.co.za/|http://www.superlinear.co.za |https://www.ambaland.com/|http://www.bonfiire.com/|http://www.radioislam.org.za|http://teeveetee.blogspot.com/|https://www.amnesty.org/en/countries/africa/south-africa/|http://theconversation.com/Africa|http://www.africaforum.org|http://www.biznews.com|http:// http://africacheck.org/|http://www.netwerk24.com|http://www.ssa.gov.za/|http://www.parliament.gov.za |http://www.rdm.co.za/|http://www.fin24.com/|http://www.nicd.ac.za/|http://www.thedailyvox.co.za|http://www.thenewage.co.za|http://www.thenewage.co.za|http://www.enca.com|http://www.capeargus.co.za|http://www.24.com/|http://www.vocfm.co.za|http://www.thoughtleader.co.za/|http://www.sundayindependent.co.za|http://www.thestar.co.za/|http://www.thepresidency.gov.za|http://www.thenewage.co.za|http://www.themediaonline.co.za|http://www.thedailymaverick.co.za/|http://www.thecitizen.co.za|http://www.techcentral.co.za/|http://www.timeslive.co.za|http://www.businessreport.co.za/|http://www.suidlanders.co.za/|http://www.sowetan.co.za|http://www.saiia.org.za/|http://www.gov.za|http://www.sanews.gov.za|http://www.sabc.co.za/news|http://www.radioislam.org.za|http://www.702.co.za|http://www.pretorianews.co.za/|http://www.politicsweb.co.za|http://www.polity.org.za|http://www.pmg.org.za/|http://www.news24.co.za|http://voices.news24.com|http://www.mjc.org.za|http://memeburn.com/|http://www.mediareviewnet.com|http://www.mg.co.za/|http://www.jmtsa.co.za|http://www.jamiat.org.za|http://www.itnewsafrica.com|http://www.iol.co.za|https://issafrica.org/|http://www.hsf.org.za|http://www.gcis.gov.za/|http://www.fm.co.za/|http://www.ewn.co.za/|http://www.engineeringnews.co.za|http://www.dieburger.com/|http://www.environment.gov.za|http://www.dirco.gov.za|http://www.da.org.za|http://www.denel.co.za|http://www.dod.mil.za|http://www.defenceweb.co.za/|http://www.dailynews.co.za/|http://www.news24.com/City-Press/|http://www.ciibroadcasting.com|http://www.ccs.org.za|http://www.capetimes.co.za/|http://www.capeargus.co.za/|http://www.businessreport.co.za/|http://www.suntimes.co.za/|http://www.businesslive.co.za|http://www.businesslive.co.za |http://www.bizcommunity.com|http://www.beeld.com|http://www.anc.org.za|http://www.alqalam.co.za/|http://www.alquds.za.net|http://www.anc.org.za|http://www.ancyl.org.za|http://constitutionallyspeaking.co.za/"

In [28]:
pullkeywords = get_keywords(r'C:\Users\Barend\Dropbox\PHD\Data\GDELT\ZA\Sources\ZASourceList.csv')
keywords = '|'.join(pullkeywords)

In [33]:
keywords = "1049.fm|2oceansviberadio.com|3d-car-shows.com|4gafrica.co.za|5stardurban.co.za|6000.co.za|702.co.za|afkinsider.com|afribiz.info|africabusiness.com|africajournalismtheworld.com|africanbusinessreview.co.za|africanmining.com|africanreview.com|africansafarisguide.com|africaoutlookmag.com|afronline.org|alumnicareers.co.za|amabhungane.co.za|anc.org.za|ann7.com|appleza.co.za|artlink.co.za|artsmart.co.za|asherworldturns.com|bbqonline.co.za|bdlive.co.za|bizcommunity.com|biznews.com|biznisafrica.co.za|bookslive.co.za|businesstech.co.za|cansa.org.za|capetalk.co.za|carletonvilleherald.com|cbn.co.za|channel24.co.za|china-africa-reporting.co.za|citizen.co.za|citypress.co.za|cnbcafrica.com|constructionreviewonline.com|creamermedia.co.za|cxpress.co.za|da.org.za|dailymaverick.co.za|dailysun.co.za|deentv.co.za|defenceweb.co.za|destinyconnect.com|destinyman.com|disrupt-africa.com|dod.mil.za|drum.co.za|dstv.com|eatout.co.za|ecr.co.za|enca.com|engineeringnews.co.za|eprop.co.za|esi-africa.com|ewn.co.za|fin24.com|finweek.com|footie.co.za|fourwaysreview.co.za|freepropertyads.co.za|gadget.co.za|gatewaynews.co.za|gearburn.com|georgeherald.com|go-dove.com|gov.za|gq.co.za|graphicrepro.co.za|graziadaily.co.za|grocotts.co.za|health-e.org.za|healthyorganic.co.za|henrileriche.com|hesp-news.org|htxt.co.za|iafrica.com|iol.co.za|issafrica.org|it-online.co.za|itinews.co.za|itnewsafrica.com|itweb.co.za|itwebafrica.com|jacarandafm.com|jamiat.org.za|jamiat.org.za|jmtsa.co.za|jobsvacancies.info|joburg.org.za|justcurious.co.za|kfm.co.za|kickoff.com|kormorant.co.za|kwevoel.co.za|lagazzettadelsudafrica.net|landbou.com|longevitylive.com|mahala.co.za|makeeverywomancount.org|marklives.com|media24.com|mediaclubsouthafrica.com|mediamonitoringafrica.org|mediaupdate.co.za|memeburn.com|mg.co.za|mgafrica.com|miningmx.com|miningreview.com|miningweekly.com|missearthsa.co.za|moneyweb.co.za|mweb.co.za|mybroadband.co.za|mygaming.co.za|mype.co.za|mzansilife.co.za|nelsonmandela.org|netwerk24.com|news24.com|newspoint.co.za|newstonight.co.za|nextvame.com|noseweek.co.za|observer.co.za|ofm.co.za|onelargeprawn.co.za|pa.org.za|penguinbooks.co.za|politicsweb.co.za|polity.org.za|portfoliocollection.com|potchefstroomherald.co.za|praag.co.za|praag.org|psl.co.za|publicnewshub.com|radioislam.org.za|rdm.co.za|rekordeast.co.za|rnews.co.za|roadsafety.co.za|sa-venues.com|sabc.co.za|sacommercialpropnews.co.za|sacreativenetwork.co.za|sagoodnews.co.za|saiia.org.za|samigration.com|sanews.gov.za|sapromo.com|saps.gov.za|sarfu.org.za|screenafrica.com|scross.co.za|sharenet.co.za|showme.co.za|son.co.za|southafrica.info|southernafricalitigationcentre.org|southerntimesafrica.com|sowetanlive.co.za|stfrancischronicle.com|sundayworld.co.za|tabloidmedia.co.za|techcentral.co.za|theafricareport.com|thedailyvox.co.za|thegremlin.co.za|themediaonline.co.za|thenewage.co.za|thepresidency.gov.za|therealjenty.com|thesouthafrica.co.za|thesouthafrican.com|thoughtleader.co.za|timeslive.co.za|ujuh.co.za|vaalweekblad.com|varsitynewspaper.co.za|ventureburn.com|wbjs.org.za|whalecottage.com|wheels24.co.za|wine.co.za|witsvuvuzela.com|women24.com|you.co.za|zalebs.com|zero2turbo.com|zoutnet.co.za|zoutpansberger.co.za|zululandobserver.co.za"

In [21]:
df_remove = testdf[~testdf['URL'].apply(str.lower).str.contains(removewords)]

In [24]:
df_key = df_remove[df_remove['SOURCEURL'].apply(str.lower).str.contains(keywords)]

In [22]:
df_remove

,URL
0,https://www.republikein.com.na/nuus/trump-op-o...
1,https://www.republikein.com.na/nuus/namibi-lie...
2,https://www.republikein.com.na/nuus/sanksies-t...
3,https://www.republikein.com.na/nuus/noord-kore...
4,https://www.republikein.com.na/nuus/trump-bevi...
5,https://www.republikein.com.na/nuus/handel-en-...
6,https://www.republikein.com.na/nuus/stilte-oor...
7,https://www.republikein.com.na/nuus/noord-kore...
8,https://www.republikein.com.na/nuus/noord-kore...
9,https://www.republikein.com.na/nuus/noord-kore...


In [25]:
df_key

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
6,694428632,20171002,201710,2017,2017.7452,AFR,AFRICA,AFR,NaN,NaN,...,NaN,4,"Nyembe, Eastern Cape, South Africa",SF,SF05,-30.4667,27.3500,-1268431,20171002,http://allafrica.com/stories/201710020625.html
14,694751339,20171003,201710,2017,2017.7479,BUS,BUSINESS,NaN,NaN,NaN,...,SF,1,South Africa,SF,SF,-30.0000,26.0000,SF,20171003,http://allafrica.com/stories/201710030707.html
15,696586071,20171010,201710,2017,2017.7671,AFR,AFRICA,AFR,NaN,NaN,...,NaN,4,"Pretoria, Gauteng, South Africa",SF,SF06,-25.7069,28.2294,-1273769,20171010,http://allafrica.com/stories/201710100285.html
16,696924217,20171011,201710,2017,2017.7699,CVL,COMMUNITY,NaN,NaN,NaN,...,NaN,4,"Makhado, Limpopo, South Africa",SF,SF09,-23.0833,29.8833,-1255499,20171011,http://allafrica.com/stories/201710110173.html
17,697243351,20171012,201710,2017,2017.7726,zul,ZULU,NaN,NaN,zul,...,NaN,4,"Gwala, Eastern Cape, South Africa",SF,SF05,-30.7807,29.8854,-1234907,20171012,http://allafrica.com/stories/201710120370.html
18,697300549,20171012,201710,2017,2017.7726,CVL,RESIDENTS,NaN,NaN,NaN,...,NaN,4,"Stellenbosch, Western Cape, South Africa",SF,SF11,-33.9346,18.8668,-1287082,20171012,http://allafrica.com/stories/201710120756.html
20,697300912,20171012,201710,2017,2017.7726,GOV,GOVERNMENT,NaN,NaN,NaN,...,NaN,4,"Cape Town, Western Cape, South Africa",SF,SF11,-33.9167,18.4167,-1217214,20171012,http://allafrica.com/stories/201710120782.html
21,698216199,20171016,201710,2017,2017.7836,REB,FREEDOM FIGHTER,NaN,NaN,NaN,...,-1272393,4,"Maritzburg, KwaZulu-Natal, South Africa",SF,SF02,-29.6168,30.3928,-1272393,20171016,http://allafrica.com/stories/201710160123.html
24,698665772,20171017,201710,2017,2017.7863,LSO,LESOTHO,LSO,NaN,NaN,...,NaN,4,"Matatiele, KwaZulu-Natal, South Africa",SF,SF02,-30.3436,28.8116,-1257922,20171017,http://allafrica.com/stories/201710170845.html
27,698633812,20171017,201710,2017,2017.7863,AFR,AFRICA,AFR,NaN,NaN,...,NaN,1,South Africa,SF,SF,-30.0000,26.0000,SF,20171017,http://allafrica.com/stories/201710170853.html


In [58]:
store = df_urls

In [24]:
df_urls = df_key.drop_duplicates(subset=['SOURCEURL'])

In [35]:
df_urls.to_csv("ZAONLYGDELTTESTAprovedSourcesOCTsmall.csv")

In [36]:
len(df_urls)

190

In [48]:
df.Body[50:120]

50     eNCA does not support the browser you are usin...
51     eNCA does not support the browser you are usin...
52     By joining with young South Africans who are s...
53     By joining with young South Africans who are s...
54     By joining with young South Africans who are s...
55     By joining with young South Africans who are s...
56     The plinth near the entrance to the ANC cemete...
57     Watch journalists speak about what the Constit...
58     The central and western parts of SA will see w...
59     The central and western parts of SA will see w...
60     The central and western parts of SA will see w...
61     The central and western parts of SA will see w...
62     The central and western parts of SA will see w...
63     The central and western parts of SA will see w...
64     The central and western parts of SA will see w...
65     The central and western parts of SA will see w...
66     The central and western parts of SA will see w...
67     The central and western 

In [23]:
df_urls = df_key

In [24]:
df_urls.iloc[0][0]

'https://www.republikein.com.na/nuus/trump-op-oorlogspad-oor-noord-korea'

In [ ]:
Testu=

In [25]:
url = df_urls.iloc[0][0]

In [26]:
PROXIES = {"http": "http://spin-use.prod.osn.dom:8080", "https": "http://spin-use.prod.osn.dom:8080"}

In [28]:
page = requests.get(url,timeout = 35).content

In [29]:
page

b'<!doctype html>\n<html lang="en">\n<head>\n<meta charset="utf-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" /><script src="/cdn-cgi/apps/head/pzKObU78JnQArzZIPof7B20BzJQ.js"></script><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(2),u=e(3),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","inlineHit","addRelease"],d="api-",l=d+"ixn-";a(p,function(e,n){s[n]=o(d+n,!0,"api")}),s.addPageActi

In [30]:
soup = bs(page, "html.parser")

In [31]:
body = soup.findAll("div")

In [32]:
body

[<div id="pre_load"><div><img src="images/loader_white1.gif"/></div></div>,
 <div><img src="images/loader_white1.gif"/></div>,
 <div class="text-right" id="my_name">Meld aan via My.na™</div>,
 <div id="menu-extra">
 <h2 class="header">
 <a href="https://www.republikein.com.na/spotprente/">Spotprente</a>
 </h2>
 <div class="swipe js-flickity overflow-main" data-flickity-options='{ "setGallerySize": false, "cellSelector":".swipe-item", "wrapAround": true, "contain":true, "lazyLoad":true, "autoPlay":4000, "pageDots":false }'><section class="swipe-item hack">
 <div class="image">
 <a class="media" href="https://d3rp5jatom3eyn.cloudfront.net/assets/images/2713/spotprent-vir-14-november-20172017-11-140.jpg" rel="spotrpente">
 <img class="lazyload" data-src="https://img.my.na/jOisYx2-s69v7OqtPeGSr2GQ_8M=/310x175/assets/images/2713/spotprent-vir-14-november-20172017-11-140.jpg" src="https://www.republikein.com.na/images/big_loader.gif">
 </img></a>
 </div>
 <h2 class="title">Spotprent vir 14 N

In [157]:
testing = str(body[8])

In [162]:
testing[46:80]

'MINISTÈRE DE L’ECONOMIE ET DES FIN'

In [33]:
paras = ["".join(x.findAll(text=True)) for x in body.findAllNext("p")]

AttributeError: 'ResultSet' object has no attribute 'findAllNext'

In [128]:
paras

['',
 '',
 '\n\nAccueil\nDjibouti &#8711;\n\n\nLa Constitution\nPrésentation Générale\nRégions\nInvestir à Djibouti\nLes Symboles\n\n\n\nLa Présidence &#8711;\n\n\nBiographie du Président\nPrérogatives du Président\nDiscours du Président\nConseil des Ministres\n\n\n\nLe Gouvernement &#8711;\n\n\nComposition\nAttributions des Ministères \n\n\n\nLes Institutions &#8711;\n\n\nAssemblée Nationale\nConseil Constitutionnel\nCommission Nationale de la Communication\nMédiateur de la République\nConseil Supérieur de la Magistrature\n\n\n\nLe Journal Officiel &#8711;\n\n\nPrésentation\nDernier Journal Officiel\nRecherche des textes\n\n\n\n<li><a href="index.html">Contact</a></li>\n\n\n',
 '\n\n\n\n\xa0\xa0\xa0\n\n\nLe Président de la République, Son Excellence Ismail Omar Guelleh, a procédé aujourd’hui, lundi, à Ali-Sabieh, Chef lieu de la région du même nom, à la mise en service officielle du projet transfrontalier d’adduction d’eau Djibouti-Ethiopie pagebreak .\r\n\xa0\r\nLa mise en service de

In [78]:
if paras == []:
    print("s")

s


In [118]:
test = all_bodies[3]

In [124]:
test

'\n\nAccueil\nDjibouti &#8711;\n\n\nLa Constitution\nPrésentation Générale\nRégions\nInvestir à Djibouti\nLes Symboles\n\n\n\nLa Présidence &#8711;\n\n\nBiographie du Président\nPrérogatives du Président\nDiscours du Président\nConseil des Ministres\n\n\n\nLe Gouvernement &#8711;\n\n\nComposition\nAttributions des Ministères \n\n\n\nLes Institutions &#8711;\n\n\nAssemblée Nationale\nConseil Constitutionnel\nCommission Nationale de la Communication\nMédiateur de la République\nConseil Supérieur de la Magistrature\n\n\n\nLe Journal Officiel &#8711;\n\n\nPrésentation\nDernier Journal Officiel\nRecherche des textes\n\n\n\n<li><a href="index.html">Contact</a></li>\n\n\n\n\n\n\n\xa0\xa0\xa0\n\n\nLe Président de la République, Son Excellence Ismail Omar Guelleh, a procédé aujourd’hui, lundi, à Ali-Sabieh, Chef lieu de la région du même nom, à la mise en service officielle du projet transfrontalier d’adduction d’eau Djibouti-Ethiopie pagebreak .\r\n\xa0\r\nLa mise en service de ce projet d’add

In [49]:
url = "http://ewn.co.za/2017/10/31/plato-we-can-t-tolerate-killings-anywhere-in-wc"

In [50]:
page = requests.get(url,timeout = 40).content
soup = bs(page, "html.parser")

In [70]:
title = soup.find("div")
whole_title = ["".join(x.findAll(text=True)) for x in title.findAllNext("h2")]
print (whole_title)


["Plato: We can't tolerate killings anywhere in WC", '\n\nPopular in Local\r\n                                \r\n                            \n', 'Comments']


In [27]:
df_urls.iloc[1][57]

'http://allafrica.com/stories/201710030707.html'

In [65]:
title_str = soup.find("h2")

In [71]:
for j in whole_title:
                title_str += str(j) + "-"
                title_str = title_str[:-1]


In [73]:
title = soup.find("title")
whole_title = ["".join(x.findAll(text=True)) for x in title.findAllNext("h1")]
            #print (whole_title)
if whole_title == []:
    title_str = soup.find("h2")
else:
    for j in whole_title:
        title_str += str(j) + "-"
        title_str = title_str[:-1]
        #print (title_str)

In [24]:
df_urls.iloc[0][0]

'http://www.lanationdj.com/mega-port-de-classe-mondiale-cofinance-chine-a-ete-officiellement-inaugure-president-de-republique-ismail-omar-guelleh-mercredi-24-mai-dernier/'

# Content Scraper

In [27]:
#With Proxy
#New Version with exceptions for GDELT or CSV Created df - Supposed to pull the body, dates and the title for each url and append them to dataframes
#Note this will also pull full HTML for page and save this, this might kill your RAM
#USe this for GDELT - df_urls.iloc[a][57] - [a][0] if you 

PROXIES = {"http": "http://spin-use.prod.osn.dom:8080", "https": "http://spin-use.prod.osn.dom:8080"}

a = 0
needhttp = 'N'

if needhttp == 'Y':
    #With http
    url = 'http://'+df_urls.iloc[a][57]
else:
    #Without http
    url = df_urls.iloc[a][57]
    
page = requests.get(url,timeout = 40, proxies = PROXIES).content
#print(url)
sent_str = ""
title_str = ""
date_str = ""
all_titles = [];
all_bodies = [];
all_dates = [];
full_pages = [];
dateempty = "Date Not Found"
titleempty ="Title Not Found"
bodyempty ="Body Not Found"

#Change the number below to pull more or less, it will give you out of bounds error if you pull to many. Do not worry about this
#Testing on smaller subset:
#while a < 29:
#Run this for full pull
while a < (len(df_urls)):
        print(a)
        print(url)
        try:
            page = requests.get(url,timeout = 40, proxies = PROXIES).content
            soup = bs(page, "html.parser")
        
        #Finding the Body Text   
            body = soup.find("body")
            paras = ["".join(x.findAll(text=True)) for x in body.findAllNext("p")]
            
            if paras == []:
                paras = ["".join(x.findAll(text=True)) for x in body.findAllNext("div")]
                for i in paras:
                    sent_str += str(i) + "-"
                    sent_str = sent_str[:-1]
                    sent_str = re.sub('[\r,\n,\xa0]', '', sent_str)
                
            else:                
                for i in paras:
                    sent_str += str(i) + "-"
                    sent_str = sent_str[:-1]               
                   
        #Finding the Title Text   
            title = soup.find("title")
            whole_title = ["".join(x.findAll(text=True)) for x in title.findAllNext("h1")]
            #print (whole_title)
            if whole_title == []:
                title_str = soup.find("h2")
            else:
                for j in whole_title:
                    title_str += str(j) + "-"
                    title_str = title_str[:-1]
                    #print (title_str)
                

        #Finding the Date with multiple options  
            date = soup.find("time")
            
            #date = str(date [58:83])
            #dates = ["".join(x.findAll(text=True)) for x in date.findAllNext("time")]
            #print(date)
            #for j in date:
                #date_str += str(j) + "-"
                #date_str = date_str[:-1] 
                #print (date_str)
                
            print (title_str)
            
            #These statements will make sure that something is filled in the columns even if it is not perfectly clean
            if title_str == '':
                all_titles.append(titleempty)
                print("Problem here title",title_str)
            else:
                all_titles.append(title_str)
                
            
            if sent_str == '':
                all_bodies.append(bodyempty)
                print("Problem here body",sent_str)
            else:
                all_bodies.append(sent_str)
                
            
            if date == None:
                all_dates.append(dateempty)
            else:
                all_dates.append(date)
            
            
            #Empty out variables
            title_str = ''
            sent_str = ''
            date = ''
            
            #Can use this to not overload websites
            #sleep(2+random.random()*9)
    
       #This will run if you pull something that is not a URL     
        except Exception as e:
            print (e)
            #a = a+1
            url = df_urls.iloc[a][57]
            #print(url)
            all_titles.append(e)
            all_bodies.append(e)
            all_dates.append(e)
            title_str = ''
            sent_str = ''
            
        #This will save the full HTML Text (BIG!!!)
        #full_pages.append(page)
        
        a = a+1
        url = df_urls.iloc[a][57]
        #url = 'http://'+df_urls.iloc[a][0]
        #print(url)

0
http://allafrica.com/stories/201710020625.html
AllAfricaSouth Africa: Soweto Gay Pride March Against Hate CrimesRelatedInFocus: South AfricaFollow AllAfricaTop Headlines: South AfricaDon't Miss
1
http://allafrica.com/stories/201710030707.html
AllAfricaZimbabwe: Mnangagwa Under Fraud ProbeRelatedInFocus: ZimbabweFollow AllAfricaTop Headlines: ZimbabweDon't Miss
2
http://allafrica.com/stories/201710100285.html
AllAfricaAfrica's Future - Seven Key TrendsRelatedInFocus: AfricaFollow AllAfricaTop Headlines: AfricaDon't Miss
3
http://allafrica.com/stories/201710110173.html
AllAfricaSouth Africa: Race to Catch Up On 19 Lost Days for Vuwani PupilsRelatedInFocus: South AfricaFollow AllAfricaTop Headlines: South AfricaDon't Miss
4
http://allafrica.com/stories/201710120370.html
AllAfricaSouth Africa: Corruption in ANC-Led Municipalities Affects Service Delivery, Moerane Commission HearsRelatedInFocus: South AfricaFollow AllAfricaTop Headlines: South AfricaDon't Miss
5
http://allafrica.com/stori

IndexError: single positional indexer is out-of-bounds

In [25]:
#Without Proxy
#New Version with exceptions for GDELT or CSV Created df - Supposed to pull the body, dates and the title for each url and append them to dataframes
#Note this will also pull full HTML for page and save this, this might kill your RAM
#USe this for GDELT - df_urls.iloc[a][57] - [a][0] if you 

a = 0
needhttp = 'N'

if needhttp == 'Y':
    #With http
    url = 'http://'+df_urls.iloc[a][0]
else:
    #Without http
    url = df_urls.iloc[a][0]
    
page = requests.get(url,timeout = 40).content
#print(url)
sent_str = ""
title_str = ""
date_str = ""
all_titles = [];
all_bodies = [];
all_dates = [];
full_pages = [];
dateempty = "Date Not Found"
titleempty ="Title Not Found"
bodyempty ="Body Not Found"

#Change the number below to pull more or less, it will give you out of bounds error if you pull to many. Do not worry about this
#Testing on smaller subset:
#while a < 29:
#Run this for full pull
while a < (len(df_urls)):
        print(a)
        print(url)
        try:
            page = requests.get(url,timeout = 40).content
            soup = bs(page, "html.parser")
        
        #Finding the Body Text   
            body = soup.find("body")
            paras = ["".join(x.findAll(text=True)) for x in body.findAllNext("p")]
            
            if paras == []:
                paras = ["".join(x.findAll(text=True)) for x in body.findAllNext("div")]
                for i in paras:
                    sent_str += str(i) + "-"
                    sent_str = sent_str[:-1]
                    sent_str = re.sub('[\r,\n,\xa0]', '', sent_str)
                
            else:                
                for i in paras:
                    sent_str += str(i) + "-"
                    sent_str = sent_str[:-1]               
                   
        #Finding the Title Text   
            title = soup.find("title")
            whole_title = ["".join(x.findAll(text=True)) for x in title.findAllNext("h1")]
            #print (whole_title)
            if whole_title == []:
                title_str = soup.find("h2")
            else:
                for j in whole_title:
                    title_str += str(j) + "-"
                    title_str = title_str[:-1]
                    #print (title_str)
                

        #Finding the Date with multiple options  
            date = soup.find("time")
            
            #date = str(date [58:83])
            #dates = ["".join(x.findAll(text=True)) for x in date.findAllNext("time")]
            #print(date)
            #for j in date:
                #date_str += str(j) + "-"
                #date_str = date_str[:-1] 
                #print (date_str)
                
            print (title_str)
            
            #These statements will make sure that something is filled in the columns even if it is not perfectly clean
            if title_str == '':
                all_titles.append(titleempty)
                print("Problem here title",title_str)
            else:
                all_titles.append(title_str)
                
            
            if sent_str == '':
                all_bodies.append(bodyempty)
                print("Problem here body",sent_str)
            else:
                all_bodies.append(sent_str)
                
            
            if date == None:
                all_dates.append(dateempty)
            else:
                all_dates.append(date)
            
            
            #Empty out variables
            title_str = ''
            sent_str = ''
            date = ''
            
            #Can use this to not overload websites
            #sleep(2+random.random()*9)
    
       #This will run if you pull something that is not a URL     
        except Exception as e:
            print (e)
            #a = a+1
            url = df_urls.iloc[a][0]
            #print(url)
            all_titles.append(e)
            all_bodies.append(e)
            all_dates.append(e)
            title_str = ''
            sent_str = ''
            
        #This will save the full HTML Text (BIG!!!)
        #full_pages.append(page)
        
        a = a+1
        url = df_urls.iloc[a][0]
        #url = 'http://'+df_urls.iloc[a][0]
        #print(url)

0
http://www.lanationdj.com/mega-port-de-classe-mondiale-cofinance-chine-a-ete-officiellement-inaugure-president-de-republique-ismail-omar-guelleh-mercredi-24-mai-dernier/
Le méga-port de classe mondiale, cofinancé par la Chine, a été officiellement inauguré par le président de la République, Ismaïl Omar Guelleh, le mercredi 24 mai dernier.

							La Nation						

1
http://www.lanationdj.com/visite-president-ismail-omar-guelleh-chine-consolidation-relations-sino-djiboutiennes-coeur-discussions/
Visite du Président Ismaïl Omar Guelleh en Chine : La consolidation des relations sino-djiboutiennes au cœur des discussions

							La Nation						

2
http://www.lanationdj.com/president-guelleh-premier-chef-detat-africain-arabe-invite-visite-officielle-chine-a-lissue-de-reelection-president-xi-jinping/
Le Président Guelleh est le premier chef d’Etat africain et arabe invité en visite officielle en Chine à l’issue de la réélection du Président Xi Jinping

							La Nation						

3
http://www.l

IndexError: single positional indexer is out-of-bounds

# Create DF with pulled Data

In [26]:
#Creating new DF
column_names = (list(df_urls.columns.values)+['Title','Body','Date','HTML'])


In [27]:
#df = pd.DataFrame(None, columns=['URL','Title','Body','Date'])
df = pd.DataFrame(None, columns=[column_names])


In [32]:
df

,URL,Title,Body,Date,HTML
0,http://www.lanationdj.com/mega-port-de-classe-...,"Le méga-port de classe mondiale, cofinancé par...","Plus grand port du pays, le Doraleh Multi-Purp...",Date Not Found,NaN
1,http://www.lanationdj.com/visite-president-ism...,Visite du Président Ismaïl Omar Guelleh en Chi...,"Le président de la République, Ismaïl Omar Gue...",Date Not Found,NaN
2,http://www.lanationdj.com/president-guelleh-pr...,Le Président Guelleh est le premier chef d’Eta...,"Le Président de la République, Ismaïl Omar Gue...",Date Not Found,NaN
3,http://www.lanationdj.com/president-guelleh-de...,« LE PRÉSIDENT GUELLEH DEVIENT LE PREMIER CHEF...,« Nos liens avec la Chine passent de rapports ...,Date Not Found,NaN
4,http://www.lanationdj.com/enjeux-vent-dest-ven...,"ENJEUX : Vent d’est, vent d’ouest\n\n\t\t\t\t\...",L’implantation récente de la première base mil...,Date Not Found,NaN
5,http://www.lanationdj.com/aboubaker-omar-hadi-...,'NoneType' object has no attribute 'findAllNext','NoneType' object has no attribute 'findAllNext','NoneType' object has no attribute 'findAllNext',NaN
6,http://www.lanationdj.com/mahmoud-ali-youssouf...,Mahmoud Ali Youssouf Ministre des Affaires étr...,"Coopération avec la Chine, regain de tension e...",Date Not Found,NaN
7,http://www.lanationdj.com/djibouticccl-lor-bla...,Djibouti/CCCL : L’or blanc du Lac Assal au cen...,"Le président de la République, Ismaïl Omar Gue...",Date Not Found,NaN
8,http://www.lanationdj.com/president-de-republi...,Le président de la République accorde une audi...,"Le président de la République, Ismaïl Omar Gue...",Date Not Found,NaN
9,http://www.presidence.dj/article.php?ID=479,None,AccueilDjibouti &#8711;La ConstitutionPrésenta...,Date Not Found,NaN


In [29]:
a = len(all_bodies)
b = len(all_titles)
c =len(all_dates)
#d =len(all_times)

In [30]:
#Adding pulled info to df
if a == b and b == c:
    df.Body = all_bodies [0:a]
    df.Title = all_titles [0:b]
    df.Date = all_dates[0:c]
    #df.Time = all_times[0:d]
else:
    sys.exit("The data did not download correctly, call Barend")

In [31]:
#Adding URLs to df
i=0
while i <len(df_urls):
    strtest = str(df_urls.iloc[i][0])
    df.URL [i] = strtest
    i = i+1

In [ ]:
#Testing
df1 = df_urls.ix[:, 0:111]

In [ ]:
#Testing
df.SOURCEURL = df_urls.ix[:, ('SOURCEURL')]

In [69]:
#Testing to see that all lists are the same length
len(df_urls)

45

In [56]:
len(full_pages)

98

In [50]:
#df.Body = all_bodies [0:100]

In [104]:
#df.Title = all_titles [0:96]

In [102]:
#df.Date = all_dates[0:745]

In [57]:
#all_titles.to_csv("testingana1.csv")

AttributeError: 'list' object has no attribute 'to_csv'

In [58]:
all_titles

["\nNews24 News. Breaking News. First\nRamaphosa apologises for 'festival of chairs' comment\r\n                    Settings\r\n                Facebook Sign-In",
 '\nNews24 News. Breaking News. First\nRamaphosa says not his job to keep Dlamini accountable\r\n                    Settings\r\n                Facebook Sign-In',
 '\nNews24 News. Breaking News. First\nOld SA flag should never be flown again - Ramaphosa\r\n                    Settings\r\n                Facebook Sign-In',
 "\nNews24 News. Breaking News. First\nZuma face-off with Ramaphosa looms over MK veterans' conference\r\n                    Settings\r\n                Facebook Sign-In",
 '\nNews24 News. Breaking News. First\nDhlomo shows support for Ramaphosa\r\n                    Settings\r\n                Facebook Sign-In',
 '\nNews24 News. Breaking News. First\nI will not serve under Ramaphosa - Ekurhuleni Mayor Mzwandile Masina\r\n                    Settings\r\n                Facebook Sign-In',
 '\nNews24 News. 

# Exports

In [33]:
df.to_csv('Djib6novto6dec.csv')

In [74]:
writer = pd.ExcelWriter('output.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

In [99]:
#Save full htmls just in case
full_htmls = pd.DataFrame(full_pages)
full_htmls.to_pickle('1FullPageNews24.pk1')

In [75]:
df.Body[25]

'\r\n                                            One student was raped, while the other was stabbed at one of the institution\'s computer labs on Monday night.\r\n                                        JOHANNESBURG - The attack on two female students on campus at the Nelson Mandela University (NMU) has sparked protests.One student was raped, while the other was stabbed with a pair of scissors when an unknown man gained entry into one of the institution\'s computer labs.The Nelson Mandela Bay Municipality has condemned the crimes.Students have now taken to the streets demanding security be stepped up at the university.The institution\'s Zandile Mbabela says: “There are parallel meetings that are running and management will be meeting with students on their demands and to work out a way in airing out those demands."Current situation at the Kraal.. Students waiting for management #NMU #NMUShutdown #NMURape pic.twitter.com/rKXtgZmuUTThe Nelson Mandela University in Port Elizabeth says it\

In [ ]:
#import sys

#sys.setrecursionlimit(2000)
df.to_pickle('PulledDataGDELT2.pk1', compression='infer')

In [ ]:
#Export file to excel 
writer = pd.ExcelWriter('fancy2.xlsx', engine='xlsxwriter')
df.to_excel(writer, index=False, sheet_name='report')
writer.save()

In [ ]:
df_urls = pd.read_csv('ZAPROTESTSEP17.csv',usecols=['SOURCEURL'],low_memory=False)

# Keyword Counter - Not Working

In [ ]:
def get_keywords(keywords_file):
    with open(keywords_file, 'r') as f:
        reader = csv.reader(f)
        keywords = list(reader)[0]
        return keywords

In [ ]:
keywords = get_keywords(r'L:\Big Data\BarendL\SomaliaKeywords.CSV')

In [ ]:
#Trying to count keywords in titles and give a total count:
total_title_hits = 0
columns = ['URL','Title','Body','Date', 'Total'] + keywords
hist_df =  pd.DataFrame(None, columns=columns)
for i in df:
    title = df[i][0]
    for idx1, keyword in enumerate(keywords):
        hits_in_title = title.lower().count(keyword)
        total_title_hits += hits_in_title
        hist_df.loc[idx1, keyword] = hits_in_title
        hist_df.loc[idx1, 'Title'] = df[i][0]
        hist_df.loc[idx1, 'Total'] = total_title_hits
hist_df = hist_df.fillna(0)